## Variational quantum eigensolver

Any qubit Hamiltonian can be decomposed into a sum of Pauli operators:
$$
H = \sum_{\boldsymbol{\alpha} \in \{0 ... 3\}^n} J_{\boldsymbol{\alpha}} \   
\sigma_{\alpha_1} \otimes \dots \otimes \sigma_{\alpha_n}, 
\quad J_{\boldsymbol{\alpha}} \in \mathbb{R}
$$

If the number of terms scales polynomially with the number of qubits, then 
$\langle \psi(\boldsymbol{\theta}) \ H \ \psi(\boldsymbol{\theta}) \rangle$ can be efficiently measured on a quantum computer: prepare the state $\psi(\boldsymbol{\theta})$ and measure each term separately. Then minimize the total energy over $\boldsymbol{\theta}$ [Peruzzo, A. et al.  Nat. Comm. 2014, 5 (1)].

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
import matplotlib.pyplot as plt
import numpy as np
from skopt import gp_minimize
from scipy.optimize import minimize
from itertools import cycle
from functools import reduce

import warnings
import sys

if not sys.warnoptions:
    warnings.simplefilter("ignore")

/home/alexey/quantumenv/lib/python3.6/site-packages/marshmallow/schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


### Tensor network states

In [2]:
class TensorNetwork:
    """
    Tensor Network states prepared on qubits
    """
    backend = "qasm_simulator"
    
    def __init__(self, q, c):
        self.backend = "qasm_simulator"
        self.q = q
        self.c = c
        self.n_qubits = len(q)
    
    def set_params(self, params):
        if self.n_params == len(params):
            self.params = params
        else:
            raise ValueError('Incorrect number of parameters!')
            
class TNWithEntangler(TensorNetwork):
    '''Tensor network that requires an entangler'''
    def __init__(self, q, c, entangler):
        super().__init__(q, c)
        if not isinstance(entangler, Entangler):
            raise TypeError('Should supply an entangler')
        self.entangler = entangler            
            
class RankOne(TensorNetwork):
    '''Tensor network for building rank-1 (unentangled) states'''
    def __init__(self, q, c):
        super().__init__(q, c)
        self.n_params = len(q) * 2
        self.params = [0] * self.n_params
        self.n_tensors = self.n_qubits

    def construct_circuit(self, params=None):
        ps = params if params is not None else self.params
        assert len(ps) == self.n_params, 'Incorrect number of parameters!'
        pc = cycle(ps)
        circ = QuantumCircuit(self.q, self.c)
        for i in range(self.n_qubits):
            circ.ry(next(pc), self.q[i])
            circ.rz(next(pc), self.q[i])
        return circ
    
class Checkerboard(TNWithEntangler):
    '''
    Tensor network where gates are applied in a checkerboard pattern
    '''

    def __init__(self, q, c, entangler, params=None, depth=4):
        super().__init__(q, c, entangler)
        self.depth = depth
        
        if self.n_qubits % 2:
            raise NotImplementedError
        if self.n_qubits % 2 == 0:
            self.n_tensors = (self.n_qubits // 2) * depth
        else:
            self.n_tensors = depth * (self.n_qubits // 2)
        self.n_params = self.n_tensors * entangler.n_params
        
        if params is None:
            self.params = [0] * self.n_params
        else:
            self.set_params(params)

    def construct_circuit(self, params=None):
        ps = params if params is not None else self.params
        assert len(ps) == self.n_params, 'Incorrect number of parameters!'
        pc = cycle(ps)
        circ = QuantumCircuit(self.q, self.c)
        def bulknext(pc):
            return ([next(pc) for j in range(self.entangler.n_params)])
        for layer in range(self.depth):
            remainder = layer % 2
            n_gates = (self.n_qubits) // 2
            for gate in range(n_gates):
                first_qubit = gate * 2 + remainder
                second_qubit = (first_qubit + 1) % self.n_qubits
                self.entangler.apply(self.q, circ, first_qubit, second_qubit, bulknext(pc))
                circ.barrier()
            circ.barrier()
        return circ    

In [3]:
class Entangler:
    '''
    Two-qubit gate used as an element of a TN state 
    '''
    def __init__(self, params=None):
        self.n_params = 2 
        if params is None:
            self.params = [0] * self.n_params
        else:
            self.set_params(params)

    def set_params(self, params):
        '''Set the parameters of the gate'''
        if len(params) == self.n_params:
            self.params = params
        else:
            raise ValueError('Incorrect amount of parameters!')
        
    def apply(self, q, circ, params=None):
        '''
        Apply with internal parameters if None
        otherwise use those supplied'''
        print('Applying parent method')
        
class IsingEntangler(Entangler):
    '''
    Entangler that uses the operators found in 
    the Ising model
    <| (exp(X) \otimes exp(X)) exp(ZZ) (exp(Z) \otimes exp(Z)) 
    '''
    def __init__(self, params=None):
        self.n_params = 5
        if params is None:
            self.params = [0] * self.n_params
        else:
            self.set_params(params)
    
    def apply(self, q, circ, i, j, params=None):
        '''
        q - QuantumRegister
        circ - QuantumCircuit
        i, j - qubits to act on
        '''
        if params is None:
            pc = cycle(self.params)
        else:
            if len(params) == self.n_params:
                pc = cycle(params)
            else:
                raise ValueError('Incorrect number of parameters!')
        circ.rx(next(pc), q[i])
        circ.rx(next(pc), q[j])
        circ.cx(q[i], q[j])
        circ.u1(next(pc), q[j])
        circ.cx(q[i], q[j])
        circ.rz(next(pc), q[i])
        circ.rz(next(pc), q[j])

### Hamiltonians

In this tutorial, we try VQE on two examples: the Ising model with transverse field and Hamiltonians embedding SAT problems.

In [4]:
X = np.array([[0, 1], [1, 0]], dtype=np.complex64)
Y = np.array([[0, -1j], [1j, 0]])
Z = np.diag([1, -1])
I = np.eye(2)

def ising_model(n_spins, J, hx):
    '''Ising model with transverse field'''
    ham = {}
    line = 'Z' + 'Z' + 'I' * (n_spins - 2)
    for i in range(n_spins):
        term = line[-i:] + line[:-i]
        ham[term] = J
    line = 'X' + 'I' * (n_spins - 1)
    if hx != 0:
        for i in range(n_spins):
            term = line[-i:] + line[:-i]
            ham[term] = hx
    return ham

def explicit_hamiltonian(ham_dict):
    n_qubits = len(list(ham_dict.keys())[0])
    #I = np.eye(2)
    #X = np.array([[0, 1], [1, 0]])
    #Y = np.array([[0, -1j], [1j, 0]])
    #Z = np.diag([1, -1])
    pauli={}
    pauli['I'] = I
    pauli['X'] = X
    pauli['Y'] = Y
    pauli['Z'] = Z
    H = np.zeros((2**n_qubits, 2**n_qubits), dtype='complex128')
    for term, energy in ham_dict.items():
        matrices=[]
        for sym in term:
            matrices.append(pauli[sym])
        total_mat = energy * reduce(np.kron, matrices)
        H +=total_mat
    return H

In [5]:
def random_clause(num_variables, k=3):
    '''Generate a random clause for k-SAT. Returns a list of three
    integers
    '''
    clause = np.random.choice(num_variables, k, replace=False) + 1
    clause = [c * ((-1)**np.random.randint(2)) for c in clause]
    return clause

def random_SAT_instance(num_variables, num_clauses, k=3):
    '''
    Generate a random instance of K-SAT
    '''
    return [random_clause(num_variables, k) for i in range(num_clauses)]

def make_SAT_Hamiltonian(sat_instance, num_bits, pauli=Z):
    '''Creates a Hamiltonian from a SAT instance. pauli specifies which
    basis to use
    '''    
    H = np.zeros((2**num_bits, 2**num_bits))
    for clause in sat_instance:
        term_list = [I] * num_bits
        for i in clause:
            term_list[abs(i) - 1] = 0.5 * (I + pauli * np.sign(i))
        H += reduce(np.kron, term_list)
    return H

### Variational quantum eigensolver

In [6]:
sv_backend = Aer.get_backend("statevector_simulator")

def measure_ham(circ, ham_dict=None, explicit_H=None, shots=1000):
    '''Measures the expected value of a Hamiltonian passed either as
    ham_dict (uses QASM backend) or as explH (uses statevector backend)
    '''
    if ham_dict is not None:
        raise NotImplementedError("Use statevector backend for now")
    elif explicit_H is not None:
        job = execute(circ, sv_backend)
        result = job.result()
        state = result.get_statevector(circ)
        state = np.array(state).reshape((len(state), 1))
        E = (state.T.conj() @ explicit_H @ state)[0,0].real
        return E
    else:
        raise TypeError('pass at least something!')
        
def any_order_VQE(TN: TensorNetwork, params_order: list,
                    init_vals=None,
                    ham_dict=None, explicit_H=None,
                    initial_circuit=None,
                    n_calls=100, verbose=True):
    '''
    Performs VQE by optimizing the tensor network in the order
    supplied in params_order.
    '''
    if init_vals:
        vals = [u for u in init_vals]
    else:
        vals = [0] * TN.n_params
    
    for free_parameters in params_order:
        print('Optimizing parameters ', free_parameters, end=' ... ')
        f = restrained_objective(TN, free_parameters, vals,
                                   explicit_H=explicit_H, ham_dict=ham_dict,
                                   initial_circuit=initial_circuit)
        suggested_point = [vals[i] for i in free_parameters]

        ## Supposedly the unitary exp(i F \theta) yields a pi-periodic
        ## cost function if F is a Pauli operator or any such that F**2 = 1
        ## Which is sadly not always the case
        res = gp_minimize(f, [(0,  2 * np.pi)] * len(free_parameters), n_calls=n_calls,
                          x0=suggested_point, verbose=verbose)

        
        #print(res.x)
        for i, n in enumerate(free_parameters):
            vals[n] = res.x[i]
        print('E = {0:0.4f}'.format(res.fun))
        #print(['{0:0.6f}'.format(v) for v in vals])
        #print(measure_ham_2(TN.construct_circuit(vals), explicit_H=explicit_H))

    return res.fun, vals
        
def restrained_objective(TN, free_parameters, default_vals, ham_dict=None, explicit_H=None, initial_circuit=None):
    '''Makes an objective function good for minimization. Locks most
    parameters, while leaving those listed in free_parameters as free
    '''

    def f(x):
        assert(len(x) == len(free_parameters)), 'Free parameters qty mismatch!'
        params = [u for u in default_vals]  ## this is probably bad
        for i, n in enumerate(free_parameters):
            params[n] = x[i]
        #print(params)
        circ = TN.construct_circuit(params)
        if initial_circuit:
            circ = initial_circuit + circ
        return measure_ham(circ, ham_dict=ham_dict, explicit_H=explicit_H)
    return f        

### Running VQE on the examples

The variational quantum eigensolver prepares a quantum state that depends on a number of parameters, samples energy of that state and uses the result as a value of the cost function. A classical optimizer then finds the next point to evaluate. In this example, the energy of the state is measured by explicitly using the Hamiltonian matrix, so it ignores the effects of finite measurement. The minimizer we use implements Bayesian optimization using Gaussian Processes.

Here we use two ansatz states: unentangled state (<tt>RankOne</tt>) and checkerboard tensor network

In [7]:
n_qubits = 6


## SAT Hamiltonian
sat_instance = random_SAT_instance(n_qubits, 23)
H = make_SAT_Hamiltonian(sat_instance, n_qubits) 

## Ising Hamiltonian
H_dict = ising_model(n_qubits, 1, -1)
H_2 = explicit_hamiltonian(H_dict)

In [8]:
q, c = QuantumRegister(n_qubits), ClassicalRegister(n_qubits)
circ = QuantumCircuit(q, c)

ent = IsingEntangler()

TN = RankOne(q, c)
n_block_pars = 2

#TN = Checkerboard(q, c, ent, depth=4)
#n_block_pars = ent.n_params

params_order = [list(range(k * n_block_pars, (k + 1) * n_block_pars))
                        for k in range(TN.n_tensors)] * 2

E, vals = any_order_VQE(TN, params_order, 
                          explicit_H=H_2, n_calls=20, 
                          verbose=False, init_vals=None)



Optimizing parameters  [0, 1] ... E = 1.7653
Optimizing parameters  [2, 3] ... E = 0.6719
Optimizing parameters  [4, 5] ... E = -1.6164
Optimizing parameters  [6, 7] ... E = -2.9814
Optimizing parameters  [8, 9] ... E = -4.9873
Optimizing parameters  [10, 11] ... E = -5.2911
Optimizing parameters  [0, 1] ... E = -5.4414
Optimizing parameters  [2, 3] ... E = -6.1748
Optimizing parameters  [4, 5] ... E = -6.2143
Optimizing parameters  [6, 7] ... E = -6.9470
Optimizing parameters  [8, 9] ... E = -7.0682
Optimizing parameters  [10, 11] ... E = -7.0682
